In [ ]:
from torchvision import transforms
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import os
import torch
import pickle
from facenet_pytorch import MTCNN
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import functional as F
import cv2

In [2]:
class FaceIDDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.images = []
        self.labels = []
        self.transform = transform if transform else transforms.Resize((64, 64))
        self.mtcnn = MTCNN(image_size=112, margin=10)
        raw_labels = []

        for label in sorted(os.listdir(root_dir)):
            folder = os.path.join(root_dir, label)
            if not os.path.isdir(folder): continue
            for img_name in os.listdir(folder):
                img_path = os.path.join(folder, img_name)
                try:
                    img = Image.open(img_path).convert("RGB")  
                    face = self.mtcnn(img)
                    if face is not None:
                        face = F.rgb_to_grayscale(face)  
                        face = self.transform(face)
                        self.images.append(face)
                        raw_labels.append(label)
                except Exception as e:
                    print(f"[ERRO] {img_path}: {e}")
                    continue

        self.le = LabelEncoder()
        self.labels = self.le.fit_transform(raw_labels)

        with open("label_encoder.pkl", "wb") as f:
            pickle.dump(self.le, f)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        face = self.images[idx]
        label = self.labels[idx]
        return face.view(-1), torch.tensor(label)

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(hidden_size, num_classes)
        )

    def forward(self, x):
        return self.model(x)

transform = transforms.Compose([
    transforms.Resize((64, 64))
])

dataset = FaceIDDataset("C:/Users/Ana Clara/Downloads/data", transform=transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=8)

input_size = 64 * 64  # GRAYSCALE
hidden_size = 128
num_classes = len(set(dataset.labels))

model = MLP(input_size, hidden_size, num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

best_val_loss = float('inf')
patience = 3
trigger_times = 0

for epoch in range(30):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for X, y in train_loader:
        X, y = X.to(device), y.to(device)
        out = model(X)
        loss = criterion(out, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(out, 1)
        correct += (predicted == y).sum().item()
        total += y.size(0)

    train_acc = 100 * correct / total

    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for X, y in val_loader:
            X, y = X.to(device), y.to(device)
            out = model(X)
            loss = criterion(out, y)
            val_loss += loss.item()
            _, predicted = torch.max(out, 1)
            val_correct += (predicted == y).sum().item()
            val_total += y.size(0)

    val_acc = 100 * val_correct / val_total
    print(f"Epoch {epoch+1} | Train Loss: {total_loss:.4f} | Train Acc: {train_acc:.2f}% | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        trigger_times = 0
        torch.save(model.state_dict(), "modelo_mlp.pth")
    else:
        trigger_times += 1
        if trigger_times >= patience:
            break


Epoch 1 | Train Loss: 5.0179 | Train Acc: 80.77% | Val Loss: 1.4519 | Val Acc: 78.57%
Epoch 2 | Train Loss: 1.3004 | Train Acc: 92.31% | Val Loss: 1.5756 | Val Acc: 78.57%
Epoch 3 | Train Loss: 0.2934 | Train Acc: 98.08% | Val Loss: 0.8806 | Val Acc: 78.57%
Epoch 4 | Train Loss: 0.2124 | Train Acc: 100.00% | Val Loss: 0.9391 | Val Acc: 78.57%
Epoch 5 | Train Loss: 0.1750 | Train Acc: 100.00% | Val Loss: 1.6644 | Val Acc: 78.57%
Epoch 6 | Train Loss: 0.0215 | Train Acc: 100.00% | Val Loss: 1.9145 | Val Acc: 78.57%


In [3]:
class ExpressaoFaceDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.images = []
        self.labels = []
        self.transform = transform if transform else transforms.Resize((64, 64))
        self.mtcnn = MTCNN(image_size=112, margin=10)
        raw_labels = []

        for label in sorted(os.listdir(root_dir)):
            folder = os.path.join(root_dir, label)
            if not os.path.isdir(folder): continue
            for img_name in os.listdir(folder):
                img_path = os.path.join(folder, img_name)
                try:
                    img = Image.open(img_path).convert("RGB")  
                    face = self.mtcnn(img)
                    if face is not None:
                        face = F.rgb_to_grayscale(face)  
                        face = self.transform(face)
                        self.images.append(face)
                        raw_labels.append(label)
                except Exception as e:
                    print(f"[ERRO] {img_path}: {e}")
                    continue

        self.le = LabelEncoder()
        self.labels = self.le.fit_transform(raw_labels)

        with open("label_encoder_exp.pkl", "wb") as f:
            pickle.dump(self.le, f)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        face = self.images[idx]
        label = self.labels[idx]
        return face.view(-1), torch.tensor(label)

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(hidden_size, num_classes)
        )

    def forward(self, x):
        return self.model(x)

transform = transforms.Compose([
    transforms.Resize((64, 64))
])

dataset = ExpressaoFaceDataset("C:/Users/Ana Clara/Downloads/expressoes", transform=transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=8)

input_size = 64 * 64 
hidden_size = 128
num_classes = len(set(dataset.labels))

model = MLP(input_size, hidden_size, num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

best_val_loss = float('inf')
patience = 3
trigger_times = 0

for epoch in range(30):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for X, y in train_loader:
        X, y = X.to(device), y.to(device)
        out = model(X)
        loss = criterion(out, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(out, 1)
        correct += (predicted == y).sum().item()
        total += y.size(0)

    train_acc = 100 * correct / total

    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for X, y in val_loader:
            X, y = X.to(device), y.to(device)
            out = model(X)
            loss = criterion(out, y)
            val_loss += loss.item()
            _, predicted = torch.max(out, 1)
            val_correct += (predicted == y).sum().item()
            val_total += y.size(0)

    val_acc = 100 * val_correct / val_total
    print(f"Epoch {epoch+1} | Train Loss: {total_loss:.4f} | Train Acc: {train_acc:.2f}% | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        trigger_times = 0
        torch.save(model.state_dict(), "modelo_expressao.pth")
    else:
        trigger_times += 1
        if trigger_times >= patience:
            break


Epoch 1 | Train Loss: 5.9514 | Train Acc: 30.56% | Val Loss: 2.3690 | Val Acc: 22.22%
Epoch 2 | Train Loss: 4.2658 | Train Acc: 61.11% | Val Loss: 2.3050 | Val Acc: 33.33%
Epoch 3 | Train Loss: 4.3069 | Train Acc: 61.11% | Val Loss: 2.3369 | Val Acc: 22.22%
Epoch 4 | Train Loss: 4.0584 | Train Acc: 66.67% | Val Loss: 2.3760 | Val Acc: 22.22%
Epoch 5 | Train Loss: 3.3082 | Train Acc: 69.44% | Val Loss: 2.0764 | Val Acc: 33.33%
Epoch 6 | Train Loss: 2.0817 | Train Acc: 86.11% | Val Loss: 1.8764 | Val Acc: 33.33%
Epoch 7 | Train Loss: 2.0606 | Train Acc: 86.11% | Val Loss: 1.7390 | Val Acc: 55.56%
Epoch 8 | Train Loss: 1.3476 | Train Acc: 94.44% | Val Loss: 1.7005 | Val Acc: 55.56%
Epoch 9 | Train Loss: 1.5983 | Train Acc: 88.89% | Val Loss: 1.9243 | Val Acc: 44.44%
Epoch 10 | Train Loss: 1.4423 | Train Acc: 83.33% | Val Loss: 2.2330 | Val Acc: 33.33%
Epoch 11 | Train Loss: 0.9793 | Train Acc: 97.22% | Val Loss: 2.4113 | Val Acc: 44.44%


In [4]:
class MLP(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, hidden_size),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.4),
            torch.nn.Linear(hidden_size, num_classes)
        )

    def forward(self, x):
        return self.model(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with open("label_encoder.pkl", "rb") as f:
    le_id = pickle.load(f)

with open("label_encoder_exp.pkl", "rb") as f:
    le_exp = pickle.load(f)

model_id = MLP(64*64, 128, len(le_id.classes_))
model_id.load_state_dict(torch.load("modelo_mlp.pth", map_location=device))
model_id.eval().to(device)

model_exp = MLP(64*64, 128, len(le_exp.classes_))
model_exp.load_state_dict(torch.load("modelo_expressao.pth", map_location=device))
model_exp.eval().to(device)

transform = transforms.Compose([
    transforms.Resize((64, 64))
])

mtcnn = MTCNN(image_size=112, margin=10, device=device)

def prever_imagem(caminho_img):
    img = Image.open(caminho_img).convert("RGB")
    face = mtcnn(img)

    face = F.rgb_to_grayscale(face)  
    face = transform(face)
    face_tensor = face.view(1, -1).to(device)

    with torch.no_grad():
        out_id = model_id(face_tensor)
        prob_id = torch.softmax(out_id, dim=1)
        conf_id, pred_id = torch.max(prob_id, dim=1)
        nome = le_id.inverse_transform([pred_id.item()])[0] if conf_id.item() > 0.4 else "Desconhecido"

    with torch.no_grad():
        out_exp = model_exp(face_tensor)
        prob_exp = torch.softmax(out_exp, dim=1)
        conf_exp, pred_exp = torch.max(prob_exp, dim=1)
        expressao = le_exp.inverse_transform([pred_exp.item()])[0] if conf_exp.item() > 0.3 else "Expressão Incerta"

    print(f"Pessoa: {nome} Expressão: {expressao} ({conf_exp.item()*100:.1f}%)")

prever_imagem(r"C:\Users\Ana Clara\Downloads\data\Ana\AnaC (2).jpeg")  


Pessoa: Ana Expressão: Feliz (61.2%)


In [ ]:
class MLP(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, hidden_size),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.4),
            torch.nn.Linear(hidden_size, num_classes)
        )

    def forward(self, x):
        return self.model(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with open("label_encoder.pkl", "rb") as f:
    le_id = pickle.load(f)

with open("label_encoder_exp.pkl", "rb") as f:
    le_exp = pickle.load(f)

model_id = MLP(64*64, 128, len(le_id.classes_))
model_id.load_state_dict(torch.load("modelo_mlp.pth", map_location=device))
model_id.eval().to(device)

model_exp = MLP(64*64, 128, len(le_exp.classes_))
model_exp.load_state_dict(torch.load("modelo_expressao.pth", map_location=device))
model_exp.eval().to(device)

mtcnn = MTCNN(keep_all=True, device=device)

transform = transforms.Compose([
    transforms.Resize((64, 64))
])

def reconhecimento_webcam():
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(img_rgb)

      
        boxes, _ = mtcnn.detect(pil_img)

        if boxes is not None:
            for box in boxes:
                x1, y1, x2, y2 = [int(b) for b in box]

                margem = 10
                x1 = max(0, x1 - margem)
                y1 = max(0, y1 - margem)
                x2 = min(frame.shape[1], x2 + margem)
                y2 = min(frame.shape[0], y2 + margem)
                face_crop = frame[y1:y2, x1:x2]

                try:
                    face_pil = Image.fromarray(cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB))
                    face_tensor = mtcnn(face_pil)
                    if face_tensor is None:
                        continue
                    face_tensor = F.rgb_to_grayscale(face_tensor)
                    face_tensor = transform(face_tensor).view(1, -1).to(device)

                    with torch.no_grad():
                        out_id = model_id(face_tensor)
                        prob_id = torch.softmax(out_id, dim=1)
                        conf_id, pred_id = torch.max(prob_id, dim=1)
                        nome = le_id.inverse_transform([pred_id.item()])[0] if conf_id.item() > 0.4 else "Desconhecido"

                    with torch.no_grad():
                        out_exp = model_exp(face_tensor)
                        prob_exp = torch.softmax(out_exp, dim=1)
                        conf_exp, pred_exp = torch.max(prob_exp, dim=1)
                        expressao = le_exp.inverse_transform([pred_exp.item()])[0] if conf_exp.item() > 0.3 else "Expressão Incerta"

                    texto = f"{nome} | {expressao}"
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, texto, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                except Exception as e:
                    print(f"Erro ao processar rosto: {e}")
                    continue

        cv2.imshow("Reconhecimento Facial + Expressão", frame)
        if cv2.waitKey(1) & 0xFF == 27:  
            break

    cap.release()
    cv2.destroyAllWindows()

reconhecimento_webcam()
